<a href="https://colab.research.google.com/github/cristianbossolasco/Inflacion-Dolar-Argentina/blob/main/Historico_Dolar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import json

In [ ]:
def get_historico_dolar(tipo, fecha_inicial, fecha_final):
  response = requests.get(f'https://mercados.ambito.com//dolar/{tipo}/grafico/{fecha_inicial}/{fecha_final}')

  data  = response.content

  # Decodificar la cadena de bytes y convertirla en una lista
  decoded_data = data.decode('utf-8')
  data_list = json.loads(decoded_data)

  # Crear el dataframe
  df = pd.DataFrame(data_list[1:], columns=data_list[0])

  # Me aseguro que la fecha tenga formato fecha
  df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')

  return df

In [ ]:
fecha_desde, fecha_hasta = '2000-01-01', '2023-06-30'

df_dolar_oficial = get_historico_dolar('formal', fecha_desde, fecha_hasta)
df_dolar_blue = get_historico_dolar('informal', fecha_desde, fecha_hasta)

# elimino duplicados
df_dolar_oficial_sin_dupl = pd.DataFrame(df_dolar_oficial.groupby('fecha')['Dólar Oficial'].max()).sort_index()
df_dolar_blue_sin_dupl = pd.DataFrame(df_dolar_blue.groupby('fecha')['Dólar Informal'].max()).sort_index()

# Utiliza el método resample para obtener el último valor de cada mes
df_dolar_oficial_mes = df_dolar_oficial_sin_dupl.resample('M').last()
df_dolar_blue_mes = df_dolar_blue_sin_dupl.resample('M').last()

# unifico los dataframe
df_dolar = df_dolar_oficial_mes.merge(
              df_dolar_blue_mes,
              left_on='fecha',
              right_on='fecha'
          )
df_dolar.rename(columns={'Dólar Oficial': 'Oficial', 'Dólar Informal' : 'Blue'}, inplace=True)
df_dolar.tail(10)

,Oficial,Blue
fecha,,
2022-09-30,154.25,288.0
2022-10-31,163.89,290.0
2022-11-30,174.55,314.0
2022-12-31,184.64,346.0
2023-01-31,194.36,383.0
2023-02-28,204.39,377.0
2023-03-31,216.24,393.0
2023-04-30,229.63,467.0
2023-05-31,250.37,493.0
